In [5]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

DATA_PATH = '../data/'
seed = 7

# Extract features used in each paper from notes.txt
with open(DATA_PATH+'notes.txt') as f:

    task_notes = f.readlines()
    task_notes = ''.join(task_notes)

    auth_features = re.search(r'(?<=91 variables:)[^.\(]*',task_notes).group()
    auth_features = re.sub('\n|\s|\"','',auth_features).split(',')

# load data
master_data = pd.read_csv(DATA_PATH+'SambnisImp.csv', index_col='X')

# split data into features used for each paper and retrieve regressand
y_master = master_data['warstds']


auth_x = master_data[auth_features]

In [6]:
#get the dataset
X = auth_x['warstds']
y = auth_x.drop('warstds', axis = 1)

# train-test split
X_train, X_test, y_train, y_test = train_test_split(y, X, test_size = 0.20)

In [7]:
#scale features; fit inly on trian split
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
from sklearn.neural_network import MLPClassifier

#defining the model
mlp = MLPClassifier(hidden_layer_sizes=(128, 128, 256, 128, 128), max_iter=1000)
mlp.fit(X_train, y_train.values.ravel())

MLPClassifier(hidden_layer_sizes=(128, 128, 256, 128, 128), max_iter=1000)

In [9]:
from sklearn.metrics import classification_report, f1_score, roc_auc_score

predictions = mlp.predict(X_test)
print(classification_report(y_test,predictions))

# ROC AUC
auc = roc_auc_score(y_test, predictions)
print('ROC AUC: %f' % auc)

f1 = f1_score(y_test, predictions)
print('F1 score: %f' % f1)





              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1403
           1       0.00      0.00      0.00        25

    accuracy                           0.98      1428
   macro avg       0.49      0.50      0.49      1428
weighted avg       0.97      0.98      0.97      1428

ROC AUC: 0.497149
F1 score: 0.000000
